In [8]:
import h5py
import numpy as np
from tqdm import tqdm
from IonSim import IonSim
from IonSim import Fit
from IonSim import Generate_TimeSeries
from IonSim import Generate_Forground

from astropy.utils.iers import conf
conf.auto_download = False

In [9]:
fg_path = '/home/wangyue/workplace/wangyue2/workplace/Ionosphere_sim/GSM2008mpi/f-30-120MHz-128-high_freq_resolution.hdf5'
#fg_path = '/home/wangyue/workplace/wangyue2/workplace/Ionosphere_sim/GSM/HI-30-120MHz-256.hdf5'
ts_path = '/home/wangyue/workplace/wangyue2/workplace/Ionosphere_sim/GSM/Time_Series_128_d10_14cnights2.hdf5'
#ts_path = '/home/wangyue/workplace/wangyue2/workplace/Ionosphere_sim/GSM/Time_Series_5p_2018_d15.hdf5'

In [10]:
#########READ DATA##########################
Tem_data = h5py.File(fg_path,'r')#GSM-syn&HI-30-120MHz_ver2.hdf5
data = Tem_data["map"]
x = np.arange(0,data.shape[1],1)
nu_list = np.arange(30,120,0.2)*1e6

time_series = h5py.File(ts_path, 'r') 
alt_collections = time_series['alt']
az_collections = time_series['az']
# lon_collections = time_series['lon']
# lat_collections = time_series['lat']
# NmD_collections = time_series['NmD']

data.shape,nu_list.shape,x.shape,alt_collections.shape,az_collections.shape

((450, 196608), (450,), (196608,), (1008, 196608), (1008, 196608))

In [11]:
def run_IonSim(ii):
    #进行模拟巡天和添加电离层效应，gaussian_beam=False使用外部beam。
    start = ii*72
    end = (ii+1)*72
    iss = IonSim.IonSim(alt = alt_collections[start:end,:],
                        az = az_collections[start:end,:],
                        nulist = nu_list,
                        nside = 128,
                        outpath = './new_signal/mpi_out_ion_2s/TimeOrder_d10_doff_128_f_rb_hfr_dvc_%s-th_cnights.hdf5'%ii,
                        #lon = lon_collections,lat = lat_collections,NmD = NmD_collections,
                       dtime = 10,
                       gaussian_beam=False,
                       dynamic_ion_complex = False,
                        dynamic_ion_d = True,
                        integral = True,
                       beam_file = '/home/wangyue/workplace/wangyue2/workplace/Ionosphere_sim/Beam_interp/interp_beam_corr_2sidelobes0d2.hdf5')
#alt,az,nulist,outpath,lon = [],lat = [],NmD = [],nside = 512,dtime = 30,gaussian_beam=True,dynamic_ion = False
    iss.run_sim(data)

In [12]:
import multiprocessing

if __name__ == '__main__':
    # Define the list to iterate over
    run_list = np.arange(0,14,1)

    # Define the number of worker processes to use
    num_processes = 14

    # Create a pool of worker processes
    pool = multiprocessing.Pool(processes=num_processes)

    # Apply the function to each item in the list using multiple processes
    results = pool.map(run_IonSim, run_list)
    

INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating normalized spherical Beam coefficient ...
INFO:root:Calculating normalized spherical Beam coefficient ...
  0%|                                                                                           | 0/450 [00:00<?, ?it/s]INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating normalized spherical Beam coefficient ...
INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating sigma of gaussian beam ...
INFO:root:Calculating sigma of gaussian beam ...
  6%|████▌                                                                            | 25/450 [00:0